In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py


In [2]:
import pandas as pd
train_df = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')


In [3]:
test_df = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv')


In [4]:
target_df = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv')


In [5]:
train_df.dropna(axis=0, how='any', inplace=True)
train_df.interpolate(method='linear', inplace=True)
test_df.interpolate(method='linear', inplace=True)
target_df.dropna(axis=0,subset=['revealed_target'], inplace=True)
target_df.interpolate(method='linear', inplace=True)



In [6]:
# !sudo pip install xgboost
from sklearn.metrics import mean_absolute_error
import numpy as np
from sklearn.model_selection import KFold

In [7]:
vals=train_df.values
y = vals[:,-3]
X=vals[:,:-3]

In [8]:
values=test_df.values
X_test=values[:,:-3]
y_test=target_df.values[:,:-2]


In [9]:
from lightgbm import early_stopping
import lightgbm as lgb
from sklearn.model_selection import KFold

kf = KFold(n_splits=3)
for train_index, test_index in kf.split(X, y):
    gbm = lgb.LGBMRegressor(objective='regression',n_estimators= 1000,
 learning_rate= 0.2861413864444475,
 num_leaves= 2420,
 max_depth= 8,
 min_data_in_leaf= 200,
 lambda_l1=100,
 lambda_l2= 30,
 min_gain_to_split= 6.843290098717733,
 bagging_fraction=0.6000000000000001,
 bagging_freq= 1,
 feature_fraction= 0.9).fit(X[train_index], y[train_index],
        eval_set=[(X[test_index], y[test_index])],
        eval_metric='l1', callbacks=[early_stopping(stopping_rounds=10)])


[LightGBM] [Warning] lambda_l1 is set=100, reg_alpha=0.0 will be ignored. Current value: lambda_l1=100
[LightGBM] [Warning] bagging_fraction is set=0.6000000000000001, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6000000000000001
[LightGBM] [Warning] min_gain_to_split is set=6.843290098717733, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=6.843290098717733
[LightGBM] [Warning] lambda_l2 is set=30, reg_lambda=0.0 will be ignored. Current value: lambda_l2=30
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[18]	valid_0's l1: 5.765	valid_0's l2: 72.20

In [10]:
import optiver2023
optiver2023.make_env.func_dict['__called__'] = False
env = optiver2023.make_env()
iter_test = env.iter_test()
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    if counter == 0:
        print(test.head(3))
        print(revealed_targets.head(3))
        print(sample_prediction.head(3))
    test_splice =  test.iloc[:, :-2]
    sample_prediction['target'] = gbm.predict(test_splice)
    env.predict(sample_prediction)
    counter += 1



This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
   stock_id  date_id  seconds_in_bucket  imbalance_size  \
0         0      478                  0      3753451.43   
1         1      478                  0       985977.11   
2         2      478                  0       599128.74   

   imbalance_buy_sell_flag  reference_price  matched_size  far_price  \
0                       -1         0.999875   11548975.43        NaN   
1                       -1         1.000245    3850033.97        NaN   
2                        1         1.000584    4359198.25        NaN   

   near_price  bid_price  bid_size  ask_price  ask_size  wap   row_id  \
0         NaN   0.999875  22940.00   1.000050   9177.60  1.0  478_0_0   
1         NaN   0.999940   1967.90   1.000601  19692.00  1.0  478_0_1   
2         NaN   0.999918   4488.22   1.000636  34955.12  1.0  478_0_2   

   currently_scored  
0             False  
1           